In [1]:
!pip3 install SPARQLWrapper

In [2]:
import requests
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
def find_wiki_id(item):
  try:
    url = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&search={item}&language=en&format=json"
    data = requests.get(url).json()
    # Return the first id (Could upgrade this in the future)
    return data['search'][0]['id']
  except:
    return None

In [ ]:
# def query_by_o(wid):
#   query = """
#       SELECT ?ssLabel ?wdLabel {
#       VALUES (?o) {(wd:""" + wid + """)}
#       ?s ?wdt ?o .
#       ?wd wikibase:directClaim ?wdt .
#       ?wd rdfs:label ?wdLabel .
#       OPTIONAL {
#       ?s rdfs:label ?sLabel .
#       FILTER (lang(?sLabel) = "en")
#       }
#       FILTER (lang(?wdLabel) = "en")
#       BIND (COALESCE(?sLabel, ?s) AS ?ssLabel)
#       }
#       ORDER BY xsd:integer(STRAFTER(STR(?wd), "http://www.wikidata.org/entity/P"))
# """
#   return query

In [4]:
def query_by_o_top(wid, count_triples):
  query = """
      SELECT ?ssLabel ?wdLabel {
      VALUES (?o) {(wd:""" + wid + """)}
      ?s ?wdt ?o .
      ?wd wikibase:directClaim ?wdt .
      ?wd rdfs:label ?wdLabel .
      OPTIONAL {
      ?s rdfs:label ?sLabel .
      FILTER (lang(?sLabel) = "en")
      }
      FILTER (lang(?wdLabel) = "en")
      BIND (COALESCE(?sLabel, ?s) AS ?ssLabel)
      }
      LIMIT """ + count_triples
  return query

In [ ]:
# def query_by_s(wid):
#   query = """
#     SELECT ?wdLabel ?ooLabel
#     WHERE {
#     VALUES (?s) {(wd:""" + wid + """)}
#     ?s ?wdt ?o .
#     ?wd wikibase:directClaim ?wdt .
#     ?wd rdfs:label ?wdLabel .
#     OPTIONAL {
#     ?o rdfs:label ?oLabel .
#     FILTER (lang(?oLabel) = "en")
#     }
#     FILTER (lang(?wdLabel) = "en")
#     BIND (COALESCE(?oLabel, ?o) AS ?ooLabel)
#     }
# """
#   return query

In [5]:
def query_by_s_top(wid, count_triples):
  query = """
    SELECT ?wdLabel ?ooLabel
    WHERE {
    VALUES (?s) {(wd:""" + wid + """)}
    ?s ?wdt ?o .
    ?wd wikibase:directClaim ?wdt .
    ?wd rdfs:label ?wdLabel .
    OPTIONAL {
    ?o rdfs:label ?oLabel .
    FILTER (lang(?oLabel) = "en")
    }
    FILTER (lang(?wdLabel) = "en")
    BIND (COALESCE(?oLabel, ?o) AS ?ooLabel)
    }
    LIMIT """ + count_triples
  return query

In [6]:
def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


# results = get_results(url, query)

# for result in results["results"]["bindings"]:
#     print(result)

In [7]:
api_url = "https://query.wikidata.org/sparql"

In [8]:
TEI_file = "/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_extraction/TEI_kg_clean.csv"
spacy_file = "/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_extraction/spacy_full.csv"
GCP_file = "/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_extraction/GCP_full_entities.csv"

In [ ]:
# for subject in spo.csv
# wikiID = find_wiki_id(subject)
# if wikiID:
  # url_by_s = query_by_s(wikiID)
  # url_by_o = query_by_o(wikiID)

  # enrichment = get_results(url_by_s)
  # for entry in enrichment["results"]["bindings"]:
  # if predicate is not uri,
    # predicate, object = entry['wdLabel']['value'],entry['ooLabel']['value']
    # generate s,p,o entry by wikidata

   # enrichment = get_results(url_by_o)
  # for entry in enrichment["results"]["bindings"]:
  # if predicate is not uri,
    # predicate, subject = entry['wdLabel']['value'],entry['ssLabel']['value']
    # generate s,p,o entry by wikidata

# same for objects in spo.csv

In [ ]:
# def generate_enrichment(root_term, s_flag = True):
#    root_id = find_wiki_id(root_term)
#    if not root_id:
#     #  print("term not found")
#      return None
#    if s_flag:
#      q = query_by_s(root_id)
#    else:
#      q = query_by_o(root_id)

#    return get_results(endpoint_url, q)
#   #  for entry in enrichment["results"]["bindings"]:
#   #     predicate, obj = entry['wdLabel']['value'],entry['ooLabel']['value']
#   #     enrich_df = pd.DataFrame(list(zip(predicate, obj)),
#   #              columns =['Predicate', 'Object'])
#   #     enrich_df['Subject'] = root_term
#   #  return enrich_df



In [9]:
N_TRIPLE = '10'
def generate_enrichment_top(root_term, s_flag):
   root_id = find_wiki_id(root_term)
   if not root_id:
    #  print("term not found")
     return None
   if s_flag:
     q = query_by_s_top(root_id, N_TRIPLE)
   else:
     q = query_by_o_top(root_id, N_TRIPLE)

   return get_results(api_url, q)
  #  for entry in enrichment["results"]["bindings"]:
  #     predicate, obj = entry['wdLabel']['value'],entry['ooLabel']['value']
  #     enrich_df = pd.DataFrame(list(zip(predicate, obj)),
  #              columns =['Predicate', 'Object'])
  #     enrich_df['Subject'] = root_term
  #  return enrich_df



### TEI enrichment, only objects, because subjects are all chapter titles

In [ ]:
# # read csv file
# import pandas as pd
# o = set()
# tb_kg = pd.read_csv(TEI_file, header=None)
# for index, row in tb_kg.iterrows():
#   # s = row[0]
#   obj = row[2]
#   o.add(obj)

In [21]:
import pandas as pd
ent = set()
tb_kg = pd.read_csv(TEI_file)

In [22]:
for index, row in tb_kg.iterrows():
  s = row[1]
  obj = row[3]
  ent.add(s)
  ent.add(obj)

In [23]:
len(tb_kg)

1290

In [24]:
len(ent)

1160

In [26]:
TEI_enrichment_path = "/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/TEI/TEI_wiki_{}.csv"
TEI_enrichment_path.format(0)

'/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/TEI/TEI_wiki_0.csv'

In [27]:
full_df = pd.DataFrame()
predicate, sub = [], []
count = 0
subset = 0
for term in ent:
  enrichment = generate_enrichment_top(term, False)
  if enrichment and 0<len(enrichment["results"]["bindings"])<=10:
    count += 1
    predicate, sub = [], []
    for entry in enrichment["results"]["bindings"]:
      # if not predicate and not obj:
      p, s = entry['wdLabel']['value'], entry['ssLabel']['value']
      # else:
      predicate.append(p)
      sub.append(s)

    if len(predicate) > 10 or len(sub) > 10:
      print("length exceed 10", term)

    enrich_df =  pd.DataFrame(list(zip(predicate, sub)),
              columns =['Predicate', 'Subject'])
    enrich_df['Object'] = term

    if full_df.empty:
      full_df = enrich_df
    else:
      full_df = pd.concat([full_df, enrich_df])

    if count > 0 and count % 100 == 0:
      full_df.to_csv(TEI_enrichment_path.format(subset))
      full_df = pd.DataFrame()
      subset += 1

full_df.to_csv(TEI_enrichment_path.format(subset))



In [28]:
full_df = pd.DataFrame()
predicate, obj = [], []
count = 0
subset += 1
for term in ent:
  enrichment = generate_enrichment_top(term, True)
  if enrichment and 0<len(enrichment["results"]["bindings"])<=10:
    count += 1
    predicate, sub = [], []

    for entry in enrichment["results"]["bindings"]:
      # if not predicate and not obj:
      p, o = entry['wdLabel']['value'], entry['ooLabel']['value']
      # else:
      predicate.append(p)
      obj.append(o)

    enrich_df =  pd.DataFrame(list(zip(predicate, obj)),
              columns =['Predicate', 'Object'])
    enrich_df['Subject'] = term

    if full_df.empty:
      full_df = enrich_df
    else:
      full_df = pd.concat([full_df, enrich_df])

    if count > 0 and count % 100 == 0:
      full_df.to_csv(TEI_enrichment_path.format(subset))
      full_df = pd.DataFrame()
      subset += 1

full_df.to_csv(TEI_enrichment_path.format(subset))



### GCP

In [11]:
import pandas as pd
gcp_kg = pd.read_csv(GCP_file)
# gcp_kg.head()
node_gcp = set()
for index, row in gcp_kg.iterrows():
  e = row[0]
  t = e.split(':')[1][:-1]
  node_gcp.add(t)

In [12]:
len(node_gcp)

9380

In [13]:
GCP_enrichment_path = "/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/GCP/GCP_wiki_{}.csv"

In [14]:
full_df = pd.DataFrame()
predicate, sub = [], []
count = 0
subset = 0
for term in node_gcp:
  enrichment = generate_enrichment_top(term, False)
  if enrichment and 0<len(enrichment["results"]["bindings"])<=10:
    count += 1
    predicate, sub = [], []
    for entry in enrichment["results"]["bindings"]:
      # if not predicate and not obj:
      p, s = entry['wdLabel']['value'], entry['ssLabel']['value']
      # else:
      predicate.append(p)
      sub.append(s)

    if len(predicate) > 10 or len(sub) > 10:
      print("length exceed 10", term)

    enrich_df =  pd.DataFrame(list(zip(predicate, sub)),
              columns =['Predicate', 'Subject'])
    enrich_df['Object'] = term

    if len(enrich_df) > 10:
      print("df length exceeds 10", term)

    if full_df.empty:
      full_df = enrich_df
    else:
      full_df = pd.concat([full_df, enrich_df])

    if count > 0 and count % 100 == 0:
      full_df.to_csv(GCP_enrichment_path.format(subset))
      full_df = pd.DataFrame()
      subset += 1

full_df.to_csv(GCP_enrichment_path.format(subset))



In [15]:
full_df = pd.DataFrame()
predicate, obj = [], []
count = 0
subset += 1
for term in node_gcp:
  enrichment = generate_enrichment_top(term, True)
  if enrichment and 0<len(enrichment["results"]["bindings"])<=10:
    count += 1
    predicate, sub = [], []

    for entry in enrichment["results"]["bindings"]:
      # if not predicate and not obj:
      p, o = entry['wdLabel']['value'], entry['ooLabel']['value']
      # else:
      predicate.append(p)
      obj.append(o)

    enrich_df =  pd.DataFrame(list(zip(predicate, obj)),
              columns =['Predicate', 'Object'])
    enrich_df['Subject'] = term

    if full_df.empty:
      full_df = enrich_df
    else:
      full_df = pd.concat([full_df, enrich_df])

    if count > 0 and count % 100 == 0:
      full_df.to_csv(GCP_enrichment_path.format(subset))
      full_df = pd.DataFrame()
      subset += 1

full_df.to_csv(GCP_enrichment_path.format(subset))

### spacy

In [16]:
# read csv file
import pandas as pd
ent = set()
tb_kg = pd.read_csv(spacy_file)
for index, row in tb_kg.iterrows():
  s = row['source']
  obj = row['target']
  ent.add(s)
  ent.add(obj)

In [17]:
len(ent)

6729

In [18]:
SPA_enrichment_path = "/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/SPA/SPA_wiki_{}.csv"

In [19]:
full_df = pd.DataFrame()
predicate, sub = [], []
count = 0
subset = 0
for term in ent:
  enrichment = generate_enrichment_top(term, False)
  if enrichment and 0<len(enrichment["results"]["bindings"])<=10:
    count += 1
    predicate, sub = [], []
    for entry in enrichment["results"]["bindings"]:
      # if not predicate and not obj:
      p, s = entry['wdLabel']['value'], entry['ssLabel']['value']
      # else:
      predicate.append(p)
      sub.append(s)

    if len(predicate) > 10 or len(sub) > 10:
      print("length exceed 10", term)

    enrich_df =  pd.DataFrame(list(zip(predicate, sub)),
              columns =['Predicate', 'Subject'])
    enrich_df['Object'] = term

    if full_df.empty:
      full_df = enrich_df
    else:
      full_df = pd.concat([full_df, enrich_df])

    if count > 0 and count % 100 == 0:
      full_df.to_csv(SPA_enrichment_path.format(subset))
      full_df = pd.DataFrame()
      subset += 1

full_df.to_csv(SPA_enrichment_path.format(subset))



In [20]:
full_df = pd.DataFrame()
predicate, obj = [], []
count = 0
subset += 1
for term in ent:
  enrichment = generate_enrichment_top(term, True)
  if enrichment and 0<len(enrichment["results"]["bindings"])<=10:
    count += 1
    predicate, sub = [], []

    for entry in enrichment["results"]["bindings"]:
      # if not predicate and not obj:
      p, o = entry['wdLabel']['value'], entry['ooLabel']['value']
      # else:
      predicate.append(p)
      obj.append(o)

    enrich_df =  pd.DataFrame(list(zip(predicate, obj)),
              columns =['Predicate', 'Object'])
    enrich_df['Subject'] = term

    if full_df.empty:
      full_df = enrich_df
    else:
      full_df = pd.concat([full_df, enrich_df])

    if count > 0 and count % 100 == 0:
      full_df.to_csv(SPA_enrichment_path.format(subset))
      full_df = pd.DataFrame()
      subset += 1

full_df.to_csv(SPA_enrichment_path.format(subset))

# Analysis - spaCy enrich

In [51]:
import glob
csvfilepath = "/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/SPA/"
csv_files = glob.glob(csvfilepath+'*.{}'.format('csv'))
csv_files

['/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/SPA/SPA_wiki_0.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/SPA/SPA_wiki_1.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/SPA/SPA_wiki_2.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/SPA/SPA_wiki_3.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/SPA/SPA_wiki_4.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/SPA/SPA_wiki_5.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/SPA/SPA_wiki_6.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/SPA/SPA_wiki_7.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/SPA/SPA_wiki_8.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/SPA/SPA_wiki_9.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/SPA/SPA_wiki_10.csv',
 '/conten

In [52]:
import pandas as pd
spacy_wiki = pd.concat([pd.read_csv(f) for f in csv_files ], ignore_index=True)
spacy_wiki.tail()

,Unnamed: 0,Predicate,Subject,Object
29556,5,has part(s),table,consumer
29557,6,described by source,table,67539
29558,7,described by source,table,http://data.thenextweb.com/tnw/entity/virtual_...
29559,8,National Historical Museums of Sweden ID,table,/g/11bc5c54l4
29560,9,BBC Things ID,table,17100386p


In [54]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_extraction/spacy_full.csv')

In [55]:
df.head()

,Unnamed: 0,source,edge,target
0,0,computing,based_on,computing paradigm ethics
1,1,multiple services,stored_on_multiple,several organizations
2,2,Systems,span,multiple security process
3,3,border,becomes,accountability
4,4,something,make,case


In [57]:
node = set()
edge = set()
for index, row in df.iterrows():
  s = row['source']
  t = row['target']
  e = row['edge']
  node.add(s)
  node.add(t)
  edge.add(e)

In [58]:
len(node), len(edge)

(6729, 2033)

In [56]:
df['edge'].describe()

count     5017
unique    2033
top        has
freq       107
Name: edge, dtype: object

In [59]:
node_spacy = set()
edge_spacy = set()
for index, row in spacy_wiki.iterrows():
  s = row['Subject']
  t = row['Object']
  e = row['Predicate']


  node_spacy.add(s)
  node_spacy.add(t)
  edge_spacy.add(e)

In [53]:
spacy_wiki.to_csv('/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/spacy_enriched.csv')

In [60]:
spacy_wiki['Predicate'].describe()

count           29561
unique           1058
top       instance of
freq             4810
Name: Predicate, dtype: object

In [62]:
diff_node = set()
for n in node_spacy:
  if n in node:
  # if n in node:
    diff_node.add(n)

In [63]:
len(node_spacy), len(edge_spacy), len(diff_node)

(13020, 1058, 1814)

In [64]:
node_enriched = node_spacy.union(node)
common_node = set()
for n in node_enriched:
  if n in node:
    common_node.add(n)
len(node_enriched), len(common_node)

(17935, 6729)

In [65]:
edge_enriched = edge_spacy.union(edge)
print(len(edge), len(edge_enriched))


2033 3081


In [72]:
common_node_tei = set()
for n in node_enriched:
  if n in tei_node:
    common_node_tei.add(n)
len(common_node_tei)

116

In [ ]:
cloud_comp1 = df[(df['source']=='cloud computing')]

In [ ]:
cloud_comp2 = df[(df['target']=='cloud computing')]


In [ ]:
cloud_comp = pd.concat([cloud_comp1, cloud_comp2])

In [ ]:
cloud_comp = cloud_comp.drop(columns='Unnamed: 0')


In [ ]:
cloud_comp.to_csv('/content/drive/MyDrive/MSCS Project/CS298_spring23/results/examples/spacy.csv')

In [ ]:
cloud_comp1 = spacy_wiki[(spacy_wiki['Subject']=='cloud computing')]

In [ ]:
cloud_comp1

,Unnamed: 0,source,edge,target
254,254,cloud computing,grouped_into_several,general categories
265,265,cloud computing,are,technological breakthroughs


In [ ]:
cloud_comp2 = spacy_wiki[(spacy_wiki['Object']=='cloud computing')]

In [ ]:
cloud_comp2

,Unnamed: 0,Predicate,Subject,Object
5049631,0,field of work,content management,cloud computing
5049632,1,named after,cacoxenite,cloud computing
5049633,2,has part(s),hospitality,cloud computing
5049634,3,subclass of,http://www.wikidata.org/entity/Q11959969,cloud computing
5049635,4,different from,expense,cloud computing
...,...,...,...,...
13253433,1539,industry,Invocation,cloud computing
13255617,3723,subclass of,Invocation,cloud computing
13256091,4197,subclass of,Invocation,cloud computing
13258204,6310,subclass of,Invocation,cloud computing


In [ ]:
cloud_comp_wiki = pd.concat([cloud_comp1, cloud_comp2])
cloud_comp_wiki = cloud_comp_wiki.drop(columns='Unnamed: 0')
cloud_comp_wiki.to_csv('/content/drive/MyDrive/MSCS Project/CS298_spring23/results/examples/spacy_wiki.csv')

In [ ]:
cloud_comp_wiki = pd.concat([cloud_comp1, cloud_comp2])
cloud_comp_wiki = cloud_comp_wiki.drop(columns='Unnamed: 0')

In [ ]:
node_spacy_wiki = set()
for index, row in cloud_comp_wiki.iterrows():
  s = row['source']
  t = row['target']

  node_spacy_wiki.add(s)
  node_spacy_wiki.add(t)

In [ ]:
lead_back=[]
for x in node_spacy_wiki:
  if x in node_spacy_wiki and x != 'cloud computing':
    lead_back.append(x)

In [ ]:
len(lead_back)

3

In [ ]:
lead_back

[nan, 'technological  breakthroughs', 'general  categories']

# Analysis - GCP enrich

In [74]:
import glob
csvfilepath = '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/GCP/'
csv_files = glob.glob(csvfilepath+'*.{}'.format('csv'))
csv_files

['/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/GCP/GCP_wiki_0.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/GCP/GCP_wiki_1.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/GCP/GCP_wiki_2.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/GCP/GCP_wiki_3.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/GCP/GCP_wiki_4.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/GCP/GCP_wiki_5.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/GCP/GCP_wiki_6.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/GCP/GCP_wiki_7.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/GCP/GCP_wiki_8.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/GCP/GCP_wiki_9.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/GCP/GCP_wiki_10.csv',
 '/conten

In [75]:
import pandas as pd
gcp_wiki = pd.concat([pd.read_csv(f) for f in csv_files ], ignore_index=True)
gcp_wiki.tail()

,Unnamed: 0,Predicate,Subject,Object
102508,5,contains the administrative territorial entity,cd,/m/05g_d6
102509,6,member of,cd,166690567
102510,7,member of,cd,doctoral thesis
102511,8,member of,cd,Victoria University of Wellington
102512,9,member of,cd,copyrighted


In [76]:
gcp_wiki = gcp_wiki.drop(columns='Unnamed: 0')


In [77]:
gcp_wiki['Predicate'].describe()

count          102513
unique           1786
top       instance of
freq            14128
Name: Predicate, dtype: object

In [ ]:
cloud_comp1 = gcp_wiki[(gcp_wiki['Subject']=='cloud computing')]
cloud_comp2 = gcp_wiki[(gcp_wiki['Object']=='cloud computing')]


In [ ]:
cloud_comp_wiki = pd.concat([cloud_comp1, cloud_comp2])
cloud_comp_wiki = cloud_comp_wiki.drop(columns='Unnamed: 0')
cloud_comp_wiki.to_csv('/content/drive/MyDrive/MSCS Project/CS298_spring23/results/examples/gcp_wiki.csv')

In [ ]:
cloud_comp_wiki.head(10)

,Predicate,Subject,Object
178501,has quality,cloud computing,Scalability
180092,has quality,cloud computing,Outputs
181683,has quality,cloud computing,71
183284,has quality,cloud computing,9.6
184885,has quality,cloud computing,information problem
186486,has quality,cloud computing,cues
188097,has quality,cloud computing,MC68000
189718,has quality,cloud computing,integers
191349,has quality,cloud computing,expectations
192988,has quality,cloud computing,1994


In [81]:
gcp_df = pd.read_csv('/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_extraction/GCP_full_entities.csv')

In [82]:
gcp_df.head()

,n
0,"{""uri"":LC}"
1,"{""uri"":abstraction}"
2,"{""uri"":clock}"
3,"{""uri"":clocks}"
4,"{""uri"":event}"


In [83]:
node_gcp = set()
for index, row in gcp_df.iterrows():
  e = row[0]
  t = e.split(':')[1][:-1]
  node_gcp.add(t)

In [84]:
gcp_node_wiki = set()
gcp_edge_wiki = set()
for index, row in gcp_wiki.iterrows():
  s = row['Subject']
  t = row['Object']
  e = row['Predicate']

  gcp_node_wiki.add(t)
  gcp_node_wiki.add(s)
  gcp_edge_wiki.add(e)


In [85]:
recognized_node_gcp = set()
for n in gcp_node_wiki:
  if n in node_gcp:
    recognized_node_gcp.add(n)

In [86]:
len(node_gcp)

9380

In [87]:
len(gcp_node_wiki), len(gcp_edge_wiki), len(recognized_node_gcp)

(41370, 1786, 6607)

In [89]:
# enriched_full_wiki_edge = gcp_edge_wiki.union(edge_gcp)
'contains' in gcp_edge_wiki

True

In [92]:
enriched_full_wiki_node = gcp_node_wiki.union(node_gcp) #unique
common_node = set()
for n in enriched_full_wiki_node:
  if n in node_gcp:
    common_node.add(n)
len(common_node), len(enriched_full_wiki_node)

(9380, 44143)

In [93]:
# tei_df = pd.read_csv('/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_extraction/TEI_kg_clean.csv')
# tei_df = tei_df.drop(columns='Unnamed: 0')

# node = set()
# # edge = set()
# for index, row in tei_df.iterrows():
#   s = row[0]
#   t = row[1]
#   e = row[2]


#   node.add(s)
#   # edge.add(t)
#   node.add(e)

increased_tei_node = set()
for n in enriched_full_wiki_node:
  if n in tei_node:
    increased_tei_node.add(n)
len(increased_tei_node)

299

In [ ]:
lead_back=[]
for x in gcp_node_wiki:
  if x in node_gcp and x != 'cloud computing':
    lead_back.append(x)


In [ ]:
len(lead_back)

5572

# Analysis - TEI

In [29]:
import glob
csvfilepath = '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/TEI/'
csv_files = glob.glob(csvfilepath+'*.{}'.format('csv'))
csv_files

['/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/TEI/TEI_wiki_0.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/TEI/TEI_wiki_1.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/TEI/TEI_wiki_2.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/TEI/TEI_wiki_3.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/TEI/TEI_wiki_4.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/TEI/TEI_wiki_5.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/TEI/TEI_wiki_6.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/TEI/TEI_wiki_7.csv',
 '/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/TEI/TEI_wiki_8.csv']

In [30]:
import pandas as pd
tei_wiki = pd.concat([pd.read_csv(f) for f in csv_files ], ignore_index=True)
tei_wiki.tail()

,Unnamed: 0,Predicate,Subject,Object
6305,5,subclass of,charges,Category:Classification
6306,6,facet of,charges,000001559
6307,7,YSO ID,charges,Category:Classification systems
6308,8,BNCF Thesaurus ID,charges,scheme
6309,9,Freebase ID,charges,classification


In [31]:
tei_wiki = tei_wiki.drop(columns='Unnamed: 0')


In [35]:
tei_wiki.to_csv('/content/drive/MyDrive/MSCS Project/results_09252023/KG_enrichment/tei_wiki.csv')

In [32]:
tei_wiki['Predicate'].describe()

count            6310
unique            432
top       instance of
freq              765
Name: Predicate, dtype: object

In [34]:
# tei_wiki = tei_wiki.drop(columns='Unnamed: 0')
tei_wiki.head()

,Predicate,Subject,Object
0,instance of,http://www.wikidata.org/entity/Q31837111,classification
1,instance of,type of map,classification
2,instance of,http://www.wikidata.org/entity/Q42168214,classification
3,instance of,brane scan,classification
4,instance of,"ESCO - European Skills, Competences, Qualifica...",classification


In [66]:
tei_df = pd.read_csv('/content/drive/MyDrive/MSCS Project/CS298_spring23/results/KG_extraction/TEI_kg_clean.csv')

In [67]:
tei_df = tei_df.drop(columns='Unnamed: 0')


In [68]:
tei_df

,0,1,2
0,introduction,contains,network-centric computing and network-centric ...
1,introduction,contains,peer-to-peer systems
2,introduction,contains,cloud computing: an old idea whose time has come
3,introduction,contains,cloud computing delivery models and services
4,introduction,contains,ethical issues in cloud computing
...,...,...,...
1285,cloud-based optimal fpga synthesis,contains,Application development
1286,cloud-based optimal fpga synthesis,contains,optimal FPGA synthesis
1287,cloud-based optimal fpga synthesis,contains,Application-specific integrated circuit (ASIC)
1288,cloud-based optimal fpga synthesis,contains,Cloud-based optimal FPGA synthesis


In [69]:
tei_node = set()
tei_edge = set()
for index, row in tei_df.iterrows():
  s = row[0]
  t = row[1]
  e = row[2]


  tei_node.add(s)
  tei_edge.add(t)
  tei_node.add(e)

In [73]:
len(tei_node), len(tei_edge)

(1160, 1)

In [41]:
node_wiki = set()
edge_wiki = set()
for index, row in tei_wiki.iterrows():
  s = row['Subject']
  t = row['Object']
  e = row['Predicate']


  node_wiki.add(s)
  node_wiki.add(t)
  edge_wiki.add(e)

In [42]:
len(node_wiki), len(edge_wiki)

(2374, 432)

In [43]:
recognized_node = set()
for n in node_wiki:
  if n in node:
    recognized_node.add(n)

In [44]:
len(recognized_node)

465

In [45]:
# enriched_full_tei_edge = original+enriching
enriched_full_tei_node = node_wiki.union(node) #unique

In [48]:
enriched_full_tei_edge = edge_wiki.union(edge)

In [49]:
len(enriched_full_tei_node), len(enriched_full_tei_edge)

(3069, 432)

In [47]:
common_node = set()
for n in enriched_full_tei_node:
  if n in node:
    common_node.add(n)
len(common_node)

1160